In [1]:
import os 

import detectors
import lightning.pytorch as pl
import timm
import torch
import torch.nn as nn
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [2]:
from models import LogSoftmaxModule, Surrogate, create_resnet18_cifar10
from utils import (choose_dataset, cifar10_normalize_values,
                   evaluate_dataloader, inverse_normalize, load_cifar10)

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

if str(device) == 'cuda:0':
    torch.set_float32_matmul_precision('high')
    print('Set precision to High.')

cuda:0
Set precision to High.


In [4]:
model = timm.create_model("resnet18_cifar10", pretrained=True)
print(isinstance(model, nn.Module))

True


In [5]:
# Testing the model. This model does not run no Softmax function.
x = torch.rand([10, 3, 32, 32])
out = model(x)
print(out[0])
# Without SoftMax
print(out.sum(1))

# With SoftMax
out = nn.functional.softmax(out, 1)
print(out.sum(1))

loss_fn = nn.KLDivLoss(reduction='batchmean')
print('[softmax] Random:', loss_fn(out[:5], out[5:]))
print('[softmax] Match:', loss_fn(out[:5], out[:5]))

# With LogSoftMax
out = nn.functional.log_softmax(out, 1)
print(out.sum(1))

loss_fn = nn.KLDivLoss(reduction='batchmean', log_target=True)
print('[log softmax] Random:', loss_fn(out[:5], out[5:]))
print('[log softmax] Match:', loss_fn(out[:5], out[:5]))

tensor([-1.5222,  0.7940, -1.5440, -2.6886,  4.1287,  0.7761, -1.7913, -1.0034,
         4.4937, -1.6448], grad_fn=<SelectBackward0>)
tensor([-0.0016, -0.0015, -0.0014, -0.0018, -0.0018, -0.0019, -0.0024, -0.0016,
        -0.0016, -0.0016], grad_fn=<SumBackward1>)
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000], grad_fn=<SumBackward1>)
[softmax] Random: tensor(-0.4261, grad_fn=<DivBackward0>)
[softmax] Match: tensor(-1.4297, grad_fn=<DivBackward0>)
tensor([-23.2368, -23.2657, -23.5822, -23.1755, -23.1490, -23.5957, -23.6005,
        -23.1891, -23.5981, -23.3978], grad_fn=<SumBackward1>)
[log softmax] Random: tensor(0.0882, grad_fn=<DivBackward0>)
[log softmax] Match: tensor(0., grad_fn=<DivBackward0>)


In [6]:
N_WORKERS = int(os.cpu_count() / 2)
BATCH_SIZE = 256

In [7]:
dataset_test = load_cifar10(train=False, require_normalize=True)
dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=N_WORKERS)

acc_test = evaluate_dataloader(model, dataloader_test)
# Expecting: 94.19%
print(f'Test accuracy: {acc_test*100:.2f}')

Files already downloaded and verified


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Test accuracy: 94.20


In [8]:
oracle = LogSoftmaxModule(model)  # The model does not output normalized outputs.
substitute = create_resnet18_cifar10()  # Using the PyTorch implementation
# Check the cell above. Note that without log function. The loss doesn't seem correct.
loss_fn = nn.KLDivLoss(reduction='batchmean', log_target=True)

In [9]:
sub_train = load_cifar10(train=True, require_normalize=True)
dataloader_train = DataLoader(sub_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=N_WORKERS)
num_training_batches = len(dataloader_train)
print("num_training_batches", num_training_batches)

Files already downloaded and verified
num_training_batches 196


In [10]:
# NOTE: `num_training_batches` is used by LRSchedular. Cannot be loaded dynamically due to a bug in PyTorch Lightning
surrogate_module = Surrogate(
    0.1,
    num_training_batches=num_training_batches,
    oracle=oracle,
    substitute=substitute,
    loss_fn=loss_fn,
    softmax=True,
)

In [11]:
MAX_EPOCHS = 50

trainer = pl.Trainer(
        max_epochs=MAX_EPOCHS,
        enable_progress_bar=True,
        logger=TensorBoardLogger("logs", name="surrogate_cifar10", default_hp_metric=False),
        callbacks=[LearningRateMonitor(logging_interval='step')],
        # fast_dev_run=True,
    )
trainer.fit(
    surrogate_module,
    train_dataloaders=dataloader_train,
    val_dataloaders=dataloader_test,
)

# Trained on original training data
acc_test = evaluate_dataloader(surrogate_module, dataloader_test)
print(f'[surrogate] Test accuracy: {acc_test*100:.2f}')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: logs/surrogate_cifar10
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params
--------------------------------------------------
0 | oracle     | LogSoftmaxModule   | 11.2 M
1 | substitute | ResNet             | 11.2 M
2 | loss_fn    | KLDivLoss          | 0     
3 | accuracy   | MulticlassAccuracy | 0     
--------------------------------------------------
11.2 M    Trainable params
11.2 M    Non-trainable params
22.3 M    Total params
89.392    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[surrogate] Test accuracy: 93.83
